# Decentralized Machine Learning

In [1]:
import warnings
warnings.simplefilter('ignore')
# warnings.filterwarnings('ignore')

### Client Setup

In [8]:
from web3 import Web3, HTTPProvider, IPCProvider
# from eth_tester import EthereumTester

# t = EthereumTester()
# print(t.get_accounts())

web3 = Web3(HTTPProvider('http://localhost:8545'))
# web3v2 = Web3(HTTPProvider('https://ropsten.infura.io/c51VIUAGmJ7HIW2QwM3J'))
# web3 = Web3(web3.currentProvider)

PASSPHRASE = 'panda'
TEST_ACCOUNT = web3.eth.coinbase

# web3.eth.blockNumber
acct = web3.personal.newAccount(PASSPHRASE)
print(acct)
print(web3.eth.getBalance(acct))
web3.eth.sendTransaction({"from": TEST_ACCOUNT, "to": acct, "value": 9999999999})
print(web3.eth.getBalance(acct))

0x196A413F38CA5d90Faa0340e5812F9D5F1C74840
0
9999999999


### Contract Deployment

In [23]:
from solc import compile_source


# CONTRACT_NAME = 'Greeter.sol'
CONTRACT_NAME = '../contracts/Query.sol'
# CONTRACT_NAME = 'Query.sol'

def compile_source_file(file_path):
   with open(file_path, 'r') as f:
      source = f.read()

   return compile_source(source)


def deploy_contract(w3, contract_interface):
    tx_hash = w3.eth.contract(
        abi=contract_interface['abi'],
        bytecode=contract_interface['bin']).deploy({"from": acct})

    address = w3.eth.getTransactionReceipt(tx_hash)['contractAddress']
    return address


def wait_for_receipt(w3, tx_hash, poll_interval):
   while True:
       tx_receipt = w3.eth.getTransactionReceipt(tx_hash)
       if tx_receipt:
         return tx_receipt
       time.sleep(poll_interval)
    
contract_source_path = CONTRACT_NAME
compiled_sol = compile_source_file(contract_source_path)

import pprint
pp = pprint.PrettyPrinter()

contract_id, contract_interface = compiled_sol.popitem()
pp.pprint(contract_interface['abi'])

web3.personal.unlockAccount(acct, PASSPHRASE)

address = deploy_contract(web3, contract_interface)
print("Deployed {0} to: {1}\n".format(contract_id, address))

contract_obj = web3.eth.contract(
   address=address,
   abi=contract_interface['abi'])

pp.pprint(contract_obj.events)

[{'constant': False,
  'inputs': [{'name': 'a', 'type': 'int256[]'},
             {'name': 'b', 'type': 'int256'}],
  'name': 'inverseScale',
  'outputs': [{'name': '', 'type': 'int256[]'}],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'constant': False,
  'inputs': [{'name': 'update', 'type': 'int256[]'},
             {'name': 'keys', 'type': 'int256[]'},
             {'name': 'numData', 'type': 'int256'}],
  'name': 'sendResponse',
  'outputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'anonymous': False,
  'inputs': [{'indexed': False, 'name': 'client', 'type': 'address'}],
  'name': 'ClientSelected',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': False, 'name': 'amount', 'type': 'uint256'}],
  'name': 'ResponseReceived',
  'type': 'event'}]
Deployed <stdin>:Query to: 0x46348c7077100BCa832922475782a0bFf8459E59



/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: deploy is deprecated in favor of contract.constructor.transact


### Event Listening

In [25]:
contract_obj = web3.eth.contract(
   address=address,
   abi=contract_interface['abi'])

response_received_event = contract_obj.events.ResponseReceived.createFilter(fromBlock='latest')
response_received_event.get_new_entries()
print(contract_obj.functions.inverseScale([5,2,3],2).call({'from': web3.eth.accounts[0]}))
contract_obj.functions.sendResponse([1,2,3], [1], 3).call({'from': web3.eth.accounts[0]})
# contract_obj.functions.sendResponse([1, 2], [1], 1).call()

# client_selected_event = contract_obj.events.ClientSelected()

def txn_digest(txn):
    return "Contract address: {0}\nEvent: {1}\nArg: {2}".format(
        txn['address'], txn['event'], txn['args'])

def event_callback(arg):
    print(txn_digest(arg))

# response_received_event.
# client_selected_event.createFilter({'filter': {'arg1':10}})

# contract_obj.events
# f = web3.utils.filters.LogFilter()
# web3.eth.filter
# f.get_all_entries()



# deploy_txn = token_contract.constructor(web3.eth.coinbase, 12345).transact()
# txn_receipt = web3.eth.getTransactionReceipt(deploy_txn)
# txn_receipt['contractAddress']

[5, 2, 3]


ValueError: {'message': 'VM Exception while processing transaction: invalid opcode', 'code': -32000, 'data': {'stack': 'c: VM Exception while processing transaction: invalid opcode\n    at Function.c.fromResults (/usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:313802)\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:537769\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:2006464\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:89955\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:75462\n    at o (/usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:78562)\n    at f (/usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:78484)\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:78286\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:89935\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:656606\n    at l (/usr/local/lib/node_modules/ga